In [ ]:
import os
import csv
import yaml
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from src import read_nz_file, read_jg_file, update_meta_data, split_df, aggregate_files, add_moving_window
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback

In [ ]:
def read_preprocessing(folder):
    X_train = pd.read_parquet(f'tmp/{folder}/X_train.parquet')
    X_test = pd.read_parquet(f'tmp/{folder}/X_test.parquet')
    y_train = pd.read_parquet(f'tmp/{folder}/y_train.parquet')['y']
    y_test = pd.read_parquet(f'tmp/{folder}/y_test.parquet')['y']

    with open(rf'./tmp/{folder}/metadata.yaml') as file:
        settings = yaml.full_load(file)

    return X_train, X_test, y_train, y_test, settings

In [ ]:
X_train, X_test, y_train, y_test, settings = read_preprocessing('basic')

In [ ]:
y_train_2 = y_train.apply(lambda x: 1 if x == 'cycling' else 0)
y_test_2 = y_test.apply(lambda x: 1 if x == 'cycling' else 0)

In [ ]:
wandb.login()

run = wandb.init(
  project="test-is-cycling",
  entity="cdl1",
  config={
    "learning_rate": 0.001,
    "epochs": 256,
    "loss_function": 'binary_crossentropy',
    "architecture": "logistic regression",
    "moving_window_size": settings['MOVING_WINDOW_SIZE'],
    "hz": settings['HZ'],
    "step_size": settings['STEP_SIZE'],
    "test_proportion": settings['TEST_PROPORTION']
  }
)

config = wandb.config

# initialize model
model = Sequential()
model.add(Dense(1, input_dim = len(X_train.columns), activation = 'sigmoid'))
model.summary()

# compile model
model.compile(loss = config.loss_function, optimizer = 'rmsprop', metrics = ['accuracy'])

model.fit(
    x = X_train,
    y = y_train_2,
    epochs = config.epochs,
    verbose  = 1,
    validation_data = (X_test, y_test_2),
    callbacks=[WandbCallback()]
)

run.finish()

In [ ]:
predictions = model.predict(X_test)

In [ ]:
pd.Series(predictions.T[0]).apply(lambda x: 1 if x >= 0.5 else 0)